In [1]:
import Simulation.EHR as ehr
import Simulation.EHR_sensitive as ehrsens
from Simulation.loaddata import load_record
import os

# Open DATA

In [2]:
filepath = 'Data/CPSC2021'
sample_set = open(os.path.join(filepath, 'RECORDS'), 'r').read().splitlines()
load_record('CPSC', sample_set[0])

(array([ 0.06019023,  0.07659802,  0.06860666, ..., -0.07251733,
        -0.06393087, -0.06444095]),
 ['non atrial fibrillation'])

# Generate EHR


In [3]:
df_EHR = ehrsens.generate_EHR('CPSC')

In [4]:
df_EHR.head(5)

,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,...,a48,a49,a50,a51,a52,a53,a54,a55,a56,a57
0,80,Female,HW,1,0. None,0. None,2. Uses,0,2119,0,...,Not Taken,Not Taken,Not Taken,Not Taken,Taken,Taken,Not Taken,Not Taken,Not Taken,Taken
1,77,Female,HW,6,1. Used,0. None,1. Used,1371,2119,0,...,Not Taken,Not Taken,Not Taken,Not Taken,Taken,Not Taken,Taken,Not Taken,Not Taken,Not Taken
2,80,Male,HW,6,2. Uses,1. Used,0. None,0,0,1411,...,Not Taken,Not Taken,Not Taken,Taken,Not Taken,Not Taken,Not Taken,Not Taken,Taken,Not Taken
3,77,Male,OW,4,1. Used,0. None,1. Used,0,2119,0,...,Not Taken,Not Taken,Not Taken,Not Taken,Not Taken,Not Taken,Not Taken,Not Taken,Not Taken,Not Taken
4,51,Female,OW,1,1. Used,1. Used,0. None,0,0,0,...,Taken,Not Taken,Not Taken,Not Taken,Not Taken,Taken,Taken,Not Taken,Not Taken,Not Taken
